# Extract top 1M pages
From here: https://dumps.wikimedia.org/other/pageviews/2024/2024-12/


In [1]:
from pathlib import Path
import os 
CWD = Path(os.getcwd())
BASEDIR = CWD.parent
asset_dir = BASEDIR / 'assets'
stats_f = asset_dir/ 'pageviews-20241201-000000'
stats_f.exists()

True

## Example from stats file:

project page_title views bytes

```

en.m Fertile_material 1 0
en.m Fertilisation 3 0
en.m Fertiliser 1 0
en.m Fertilisers_and_Chemicals_Travancore 1 0
en.m Fertility 10 0
en.m Fertility_and_intelligence 4 0
en.m Fertility_and_religion 2 0
en.m Fertility_awareness 2 0
en.m Fertility_clinic 1 0
en.m Fertility_fraud 5 0
en.m Fertility_goddess 1 0
en.m Fertility_in_art 2 0
en.m Fertility_preservation 1 0

```

In [2]:
import pandas as pd

# Replace 'filename.txt' with your actual file path
df = pd.read_csv(stats_f, sep=' ', header=None, names=['project', 'page_title', 'views', 'bytes'])

print(df.head())


  project                              page_title  views  bytes
0     NaN                                       -      1      0
1     NaN             Category:Project_concept/ss      1      0
2     NaN                      File:En_obs_v8.pdf      1      0
3     NaN             File:Wikifunctions-logo.svg      1      0
4     NaN  Special:MyLanguage/Wikifunctions:About      1      0


In [4]:
english_mask = (df["project"] == "en") | (df["project"] == "en.m")
output_f = asset_dir / 'english_pageviews.csv'
if not output_f.exists():
    english_df = df[english_mask]


    english_df.head()

    english_df = english_df.groupby('page_title', as_index=False).agg({'views': 'sum', 'bytes': 'sum'})


    # sort by views
    english_df = english_df.sort_values(by='views', ascending=False)
    # combine views from en and en.m

    # for page ttles swap _ wit " "
    english_df['page_title'] = english_df['page_title'].str.replace('_', ' ')

    # drop bytes col
    english_df = english_df.drop(columns=['bytes'])
    # save to asset_dir
    english_df.to_csv(output_f, index=False)
    
    print(f"saved to {output_f}")   
else:
    english_df = pd.read_csv(output_f)
    print(f"loaded from {output_f}")
print(english_df.shape)
english_df.head(20)


loaded from /n/home04/rrinberg/code/wiki-rag/assets/english_pageviews.csv
(1678755, 2)


,page_title,views
0,Special:Search,207123
1,Main Page,175514
2,Survivor Series: WarGames (2024),36908
3,Wikipedia:Featured pictures,20938
4,Wicked (2024 film),17021
5,Wicked (musical),14723
6,Bob Bryar,13799
7,Moana 2,11342
8,Charles Kushner,11295
9,Gladiator II,11059


## check the top 1M webapages against the wiki pages that are saved in the wikipedia

In [67]:
# clean titles 
from importlib import reload 
from wiki_rag import wikipedia
reload(wikipedia)

clean_title = wikipedia.clean_title 


import json
def save_json(d, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(d, f, ensure_ascii=False, indent=4)
        
        
import pickle
def save_pickle(d, filepath):
    import pickle
    with open(filepath, 'wb') as f:
        pickle.dump(d, f)


In [82]:
# wiki titles from 
data_cache= Path("/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia")
import wikiextractor

dir(wikiextractor)
wikiextractor.extract.Extractor
  
json_dir = data_cache / 'json'
# read line by line, extracting json from wiki_00
import json
wiki_titles = []

title_to_file_path = {}
# recursive glob in json_dir.glob('**/wiki_*')
jsons_ = list(json_dir.glob('**/wiki_*'))
print(len(jsons_))
wiki_00 = json_dir / 'AA/wiki_00'

title_to_file_path_f = asset_dir / 'title_to_file_path.json'
title_to_file_path_f_pkl = asset_dir / 'title_to_file_path.pkl'

from tqdm import tqdm 

if True: #title_to_file_path_f_pkl.exists():
    with open(title_to_file_path_f_pkl, 'rb') as f:
        title_to_file_path = pickle.load(f)
    
else:
    for path in tqdm(jsons_):
        with open(path, 'r', encoding='utf-8') as f:
            for line_number, line in enumerate(f, start=1):
                try:
                    data = json.loads(line)
                    title = data['title']
                    title = clean_title(title)
                    # Save both file path and line number
                    title_to_file_path[title] = (str(path), line_number)
                except json.JSONDecodeError:
                    continue
    print("saving json")
    save_json(title_to_file_path, title_to_file_path_f)
    print(f"saved json")
    save_pickle(title_to_file_path, title_to_file_path_f_pkl)
    

19103


100%|██████████| 19103/19103 [01:49<00:00, 174.33it/s]


saving json
saved json


In [84]:
title_to_file_path.get("AfghanistanCommunications")

def read_line_from_file(path, line_number):
    """Read a specific line (1-indexed) from a text file."""
    with open(path, 'r', encoding='utf-8') as f:
        for current_line_number, line in enumerate(f, start=1):
            if current_line_number == line_number:
                return line
    raise ValueError(f"Line number {line_number} not found in file: {path}")


def get_wiki_page(title, title_to_file_path):
    path, row_num = title_to_file_path.get(title, (None, None))
    if path is None:
        return None
    # read the json file, and find the line with the title
    data = read_line_from_file(path, row_num)
    return json.loads(data)


example_page = get_wiki_page("AfghanistanCommunications", title_to_file_path)
"SurvivorSeriesWarGames" in title_to_file_path
example_page = get_wiki_page("survivorserieswargames", title_to_file_path)
example_page

{'id': '71794564',
 'revid': '7471087',
 'url': 'https://en.wikipedia.org/wiki?curid=71794564',
 'title': 'Survivor Series WarGames',
 'text': ''}

In [71]:
example_page

In [72]:
c = 0
for k,v in title_to_file_path.items():
    print(k, v)
    c+=1
    if c>100:
        break

accessiblecomputing /n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AK/wiki_42
anarchism /n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00
afghanistanhistory /n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/EQ/wiki_10
afghanistangeography /n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00
afghanistanpeople /n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00
afghanistancommunications /n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00
afghanistantransportations /n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00
afghanistanmilitary /n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00
afghanistantransnationalissues /n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00
assistivetechnology /n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AL/wiki_62
amoeboidtaxa /n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00
autismspectrum /n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/FC/

In [73]:
title_to_file_path

{'accessiblecomputing': '/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AK/wiki_42',
 'anarchism': '/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00',
 'afghanistanhistory': '/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/EQ/wiki_10',
 'afghanistangeography': '/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00',
 'afghanistanpeople': '/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00',
 'afghanistancommunications': '/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00',
 'afghanistantransportations': '/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00',
 'afghanistanmilitary': '/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00',
 'afghanistantransnationalissues': '/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki_00',
 'assistivetechnology': '/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AL/wiki_62',
 'amoeboidtaxa': '/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/json/AA/wiki

In [74]:
import re
for i, row in english_df.iterrows():
    
    title = row['page_title']
    print(title)
    title = clean_title(title)
    print(f"new title {title}")
    if False:
        date_clean_title = re.sub(r'\s*\(\d{4}\)', '', title) # remove date at the end
        print(f"date_clean_title: '{date_clean_title}'")
        page = get_wiki_page(title, title_to_file_path)
    
    page = get_wiki_page(title, title_to_file_path)
            
    if page is not None:
        print(f"Title: {page['title']}")
        print(f"URL: {page['url']}")
        print(f"Views: {row['views']}")
        print()
    else:
        print(f"Page not found for title: {title}")
    if i > 10:
        break

Special:Search
new title specialsearch
Page not found for title: specialsearch
Main Page
new title mainpage
Title: MAINPAGE
URL: https://en.wikipedia.org/wiki?curid=43620591
Views: 175514

Survivor Series: WarGames (2024)
new title survivorserieswargames
Title: Survivor Series WarGames
URL: https://en.wikipedia.org/wiki?curid=71794564
Views: 36908

Wikipedia:Featured pictures
new title wikipediafeaturedpictures
Page not found for title: wikipediafeaturedpictures
Wicked (2024 film)
new title wicked(2024film)
Title: Wicked (2024 film)
URL: https://en.wikipedia.org/wiki?curid=60455274
Views: 17021

Wicked (musical)
new title wicked(musical)
Title: Wicked (musical)
URL: https://en.wikipedia.org/wiki?curid=1678965
Views: 14723

Bob Bryar
new title bobbryar
Title: Bob Bryar
URL: https://en.wikipedia.org/wiki?curid=50782727
Views: 13799

Moana 2
new title moana2
Title: Moana 2
URL: https://en.wikipedia.org/wiki?curid=65419058
Views: 11342

Charles Kushner
new title charleskushner
Title: Charl

In [63]:
for key in title_to_file_path.keys():
    if ("Survivor" in key) and ("WarGames" in key):
        print(key) 

WWESurvivorSeriesWarGames
SurvivorSeriesWarGames


# Extract the top 1M posts for the RAG, that are have wiki facts about them


In [86]:
counts = 0
for i, row in tqdm(english_df.iterrows()):
    title = clean_title(row['page_title'])
    data = get_wiki_page(title, title_to_file_path)
    if data is None:
        continue
    text = data.get('text', "")
    if len(text) < 100:
        continue
    if data is not None:
        counts += 1
    if i > 100_000:
        break 
print(f"{counts} pages found; out of {i}")

100001it [01:06, 1505.20it/s]

66761 pages found; out of 100001


In [78]:
data

{'id': '60457775',
 'revid': '48633785',
 'url': 'https://en.wikipedia.org/wiki?curid=60457775',
 'title': 'Sing 2',
 'text': 'Sing 2 is a 2021 American animated jukebox musical comedy film produced by Universal Pictures and Illumination, and distributed by Universal. The sequel to "Sing" (2016), it was written and directed by Garth Jennings, co-directed by Christophe Lourdelet, and produced by Chris Meledandri and Janet Healy. The story follows Buster Moon and his group putting on a show in Redshore City while working to impress an entertainment mogul and enlist a reclusive rock star to perform with the group.\nThe film stars an ensemble voice cast consisting of Matthew McConaughey, Reese Witherspoon, Scarlett Johansson, Taron Egerton, Tori Kelly, Nick Kroll, Jennings, Peter Serafinowicz, Jennifer Saunders, and Nick Offerman, reprising their roles from the first film. The sequel also features new characters voiced by Bobby Cannavale, Bono, Halsey, Pharrell Williams, Chelsea Peretti, L